In [194]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import stats
from scipy.stats import linregress
import seaborn as sns
import datetime as dt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from joblib import dump, load

sns.set()

In [195]:
data = pd.read_csv("clean_bike_data.csv")
data['DATETIME'] =  pd.to_datetime(data['DATETIME'], infer_datetime_format=True)
data = data[(data["DATETIME"].dt.month >= 6) & (data["DATETIME"].dt.month < 10)]
data = data.dropna()
data

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,LAT,LON,DATETIME
454,Theft of Bicycle,2003,6,1,0,0,10XX MARINASIDE CRES,Central Business District,49.273416,-123.117159,2003-06-01 00:00:00
455,Theft of Bicycle,2003,6,1,1,0,38XX CAMBIE ST,Riley Park,49.251056,-123.115267,2003-06-01 01:00:00
456,Theft of Bicycle,2003,6,1,10,0,10XX COAL HARBOUR RD,Central Business District,49.288912,-123.116788,2003-06-01 10:00:00
457,Theft of Bicycle,2003,6,1,15,0,CAROLINA ST / E 8TH AVE,Mount Pleasant,49.263438,-123.091025,2003-06-01 15:00:00
458,Theft of Bicycle,2003,6,1,16,0,10XX BEACH AVE,West End,49.276572,-123.134358,2003-06-01 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...
30837,Theft of Bicycle,2019,9,30,18,0,19XX W 6TH AVE,Kitsilano,49.266301,-123.148165,2019-09-30 18:00:00
30838,Theft of Bicycle,2019,9,30,20,30,27XX MAIN ST,Mount Pleasant,49.260925,-123.100910,2019-09-30 20:30:00
30839,Theft of Bicycle,2019,9,30,21,0,37XX PRINCE EDWARD ST,Riley Park,49.251549,-123.096598,2019-09-30 21:00:00
30840,Theft of Bicycle,2019,9,30,21,0,18XX W 1ST AVE,Kitsilano,49.270711,-123.145691,2019-09-30 21:00:00


In [196]:
le_neigh = preprocessing.LabelEncoder()
le_block = preprocessing.LabelEncoder()
le_weath = preprocessing.LabelEncoder()

df_class = data.copy(deep=True)
df_class["NEIGHBOURHOOD"] = df_class["NEIGHBOURHOOD"].apply(lambda x: str(x))
df_class["HUNDRED_BLOCK"] = df_class["HUNDRED_BLOCK"].apply(lambda x: str(x))
# df_class["WEATHER"] = df_class["WEATHER"]


le_neigh.fit(df_class["NEIGHBOURHOOD"])
df_class["NEIGHBOURHOOD"] = le_neigh.transform(df_class["NEIGHBOURHOOD"])

le_block.fit(df_class["HUNDRED_BLOCK"])
df_class["HUNDRED_BLOCK"] = le_block.transform(df_class["HUNDRED_BLOCK"])

# le_weath.fit(df_class["WEATHER"])
# df_class["WEATHER"] = le_weath.transform(df_class["WEATHER"])

In [197]:
X = df_class[["MONTH", "DAY", "NEIGHBOURHOOD", "HUNDRED_BLOCK"]]
y = df_class["HOUR"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.80)

In [199]:
model = DecisionTreeClassifier(max_depth=1)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))

p_test = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

tuning = GridSearchCV(estimator=DecisionTreeClassifier(),
                      param_grid = p_test,
                      scoring='accuracy',
                      n_jobs=6,
                      cv=5,
                      iid=False)

tuning.fit(X_train, y_train)
print(tuning.best_params_)

0.08129344212753098
0.07666742201072589
{'max_depth': 1}


In [201]:
model = RandomForestClassifier(n_estimators=30)
model.fit(X_train, y_train)

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))

p_test = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

tuning = GridSearchCV(estimator=DecisionTreeClassifier(),
                      param_grid = p_test,
                      scoring='accuracy',
                      n_jobs=6,
                      cv=5,
                      iid=False)

tuning.fit(X_train, y_train)
print(tuning.best_params_)

0.9879117558174675
0.06216481607372158
{'max_depth': 1}


In [190]:
model1 = GaussianNB()
model1.fit(X_train, y_train)

print(model1.score(X_train, y_train))
print(model1.score(X_valid, y_valid))

0.061650045330915684
0.0410907168215122


In [191]:
model4 = GradientBoostingClassifier()
model4.fit(X_train, y_train)

dump(model4, 'bike_theft_prediction_model.joblib') 
print(model4.score(X_train, y_train))
print(model4.score(X_valid, y_valid))

0.6007857358718646
0.03542563637736989


In [ ]:
model = load('bike_theft_prediction_model.joblib') 

print(model.score(X_train, y_train))
print(model.score(X_valid, y_valid))